https://www.bbci.de/competition/ii/albany_desc/albany_desc_i.html

The dataset is from the above website. <br>
**NOTE: This is done just for understanding the singal processing and Fourier Transform. Not for competition purpose**

In [57]:
import urllib
import shutil
import os
import numpy as np
from scipy.io import loadmat

In [58]:
#download the dataset and store is in /kaggle/working/ directory
urllib.request.urlretrieve('https://www.bbci.de/competition/download/competition_ii/albany/albany_data_AA.zip', 'DataA.zip')

('DataA.zip', <http.client.HTTPMessage at 0x7860e9213390>)

In [59]:
# unzip
shutil.unpack_archive('DataA.zip', '/kaggle/working/Data')

In [60]:
# Examining the available features
data =loadmat('/kaggle/working/Data/AA001.mat')
print(data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'run', 'trial', 'sample', 'signal', 'TargetCode', 'ResultCode', 'StimulusTime', 'Feedback', 'IntertrialInterval', 'Active', 'SourceTime', 'RunActive', 'Recording', 'IntCompute', 'Running'])


In [61]:
all_files = os.listdir('/kaggle/working/Data')
sorted(all_files)[:6]

['AA001.mat', 'AA002.mat', 'AA003.mat', 'AA004.mat', 'AA005.mat', 'AA006.mat']

In [62]:
# each session 6 runs
# each run 32 trials
# total trails per session = 6 * 32

In [91]:
X_train = []
y_train = []

for file in sorted(all_files)[:6]:
    file_path = os.path.join('/kaggle/working/Data', file)
    data = loadmat(file_path)

    signal = data['signal']
    trial = data['trial']
    Feedback = data['Feedback']
    TargetCode = data['TargetCode']
    # TargetCode 0 ->
    unique_trials = np.unique(trial[(TargetCode>0).flatten()])
    n_trials = len(unique_trials)

    Xx = np.zeros((n_trials, 128))
    yy = np.zeros(n_trials)
    for i, trial_num in enumerate(unique_trials):
        trial_mask = (trial== trial_num) & (Feedback==1)
        trial_signal = signal[trial_mask.flatten(), :]

        mu = []
        beta = []
        for ch in range(64): #64 channels
            channel_signal = trial_signal[:, ch]
            fft_values = np.fft.rfft(channel_signal)
            fft_freq = np.fft.rfftfreq(len(channel_signal), d=1/160) #160 Hz 
            # power = amplitude^2
            # Mu range 8-12 Hz, 
            mu_idx = (fft_freq >= 8) & (fft_freq <= 12)
            mu_power = np.sum(np.abs(fft_values[mu_idx])**2)
            mu.append(np.log(mu_power + 1e-10))
            # Beta range 18-24 Hz
            beta_idx = (fft_freq >=18) & (fft_freq <=24)
            beta_power = np.sum(np.abs(fft_values[beta_idx])**2)
            beta.append(np.log(beta_power+1e-10))
        Xx[i, :] = np.concatenate([mu, beta])
        yy[i] = TargetCode[trial_mask.flatten()][0]
    X_train.append(Xx)
    y_train.append(yy)
X = np.vstack(X_train)
y = np.concatenate(y_train)
print(f"X shape: {X.shape}, y shape: {y.shape}")

/tmp/ipykernel_37/3221617093.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  yy[i] = TargetCode[trial_mask.flatten()][0]


X shape: (1152, 128), y shape: (1152,)


In [68]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

In [72]:
# Train SVM
clf = SVC(kernel='rbf', C=1.0, gamma='scale')

scores = cross_val_score(clf, X, y, cv=5)
print(f"Cross-validation accuracy: {scores.mean():.3f} (+/- {scores.std():.3f})")

# Fit on all training data
clf.fit(X, y)
print("Model trained!")

# Check what baseline was
print("Baseline to beat: ~60-80% (from the competition)")

Cross-validation accuracy: 0.418 (+/- 0.018)
Model trained!
Baseline to beat: ~60-80% (from the competition)


In [46]:
target_data = loadmat('/kaggle/working/Data/AA007.mat')
target_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'run', 'trial', 'sample', 'signal', 'TargetCode', 'ResultCode', 'StimulusTime', 'Feedback', 'IntertrialInterval', 'RestPeriod', 'Active', 'SourceTime', 'RunActive', 'Recording', 'IntCompute', 'Running', 'idxs'])

In [74]:
sorted(all_files)[6:]

['AA007.mat', 'AA008.mat', 'AA009.mat', 'AA010.mat']

In [92]:
X_test = []

for file in sorted(all_files)[6:]:
    file_path = os.path.join('/kaggle/working/Data', file)
    data = loadmat(file_path)

    signal = data['signal']
    trial = data['trial']
    Feedback = data['Feedback']
    TargetCode = data['TargetCode']

    # Its given that the targetCode were given values -1(unknown)
    unique_trials = np.unique(trial[(TargetCode==-1).flatten()])
    n_trials = len(unique_trials)
    Xx = np.zeros((n_trials, 128))
    for i, trial_num in enumerate(unique_trials):
        trial_mask = (trial== trial_num) & (Feedback==1)
        trial_signal = signal[trial_mask.flatten(), :]

        mu = []
        beta = []
        
        for ch in range(64): # 64 channels
            channel_signal = trial_signal[:, ch]
            fft_values = np.fft.rfft(channel_signal)
            fft_freq = np.fft.rfftfreq(len(channel_signal), d=1/160)

            mu_idx = (fft_freq >= 8) & (fft_freq <= 12)
            # power = amplitude^2
            mu_power = np.sum(np.abs(fft_values[mu_idx])**2)
            mu.append(np.log(mu_power + 1e-10))

            beta_idx = (fft_freq >=18) & (fft_freq <=24)
            beta_power = np.sum(np.abs(fft_values[beta_idx])**2)
            beta.append(np.log(beta_power+1e-10))
        Xx[i, :] = np.concatenate([mu, beta])
    X_test.append(Xx)
X2 = np.vstack(X_test)
print(f"X shape: {X2.shape}")

X shape: (768, 128)


In [81]:
pred = clf.predict(X2)
pred

array([1., 4., 4., 1., 2., 2., 1., 2., 4., 2., 2., 4., 2., 2., 2., 2., 4.,
       1., 4., 2., 4., 4., 2., 3., 3., 2., 2., 1., 4., 2., 1., 4., 2., 1.,
       1., 1., 2., 4., 4., 1., 4., 3., 1., 1., 1., 1., 1., 1., 1., 1., 4.,
       2., 1., 4., 1., 1., 2., 2., 4., 1., 4., 2., 1., 1., 4., 2., 2., 1.,
       3., 2., 1., 2., 1., 2., 2., 2., 1., 2., 4., 4., 2., 1., 2., 1., 1.,
       1., 1., 1., 2., 2., 4., 1., 2., 1., 2., 4., 2., 1., 1., 1., 4., 1.,
       2., 4., 4., 1., 4., 2., 4., 1., 1., 1., 1., 1., 1., 4., 4., 2., 4.,
       4., 1., 2., 1., 1., 4., 2., 2., 4., 4., 1., 4., 1., 4., 2., 1., 1.,
       1., 4., 2., 4., 2., 1., 4., 4., 4., 4., 2., 2., 4., 3., 2., 4., 4.,
       1., 2., 2., 2., 2., 4., 3., 1., 1., 1., 1., 1., 1., 4., 2., 4., 4.,
       2., 2., 4., 4., 1., 4., 1., 4., 1., 1., 2., 2., 4., 4., 4., 4., 4.,
       4., 4., 1., 4., 4., 1., 2., 1., 3., 1., 1., 2., 3., 2., 4., 4., 1.,
       2., 1., 4., 4., 1., 1., 1., 4., 2., 4., 4., 1., 1., 4., 2., 1., 3.,
       1., 1., 1., 1., 1.

In [83]:
from sklearn.ensemble import RandomForestClassifier

In [93]:
X, y

(array([[22.58959041, 22.58950942, 22.52536577, ..., 21.55112087,
         21.76166795, 21.53142994],
        [22.68442937, 22.80662823, 22.67833077, ..., 21.94494565,
         22.10126097, 21.90911708],
        [22.65463226, 22.55100783, 22.64499938, ..., 22.0814958 ,
         22.09569354, 22.06887755],
        ...,
        [23.49670174, 23.5020176 , 23.71470202, ..., 22.39494854,
         22.18357974, 22.27112924],
        [22.6131923 , 22.86009532, 23.11649828, ..., 22.58316378,
         22.55225916, 22.52870064],
        [23.1381275 , 23.39401083, 23.32461794, ..., 21.9697151 ,
         21.82878527, 22.04953041]]),
 array([2., 3., 4., ..., 1., 4., 2.]))

In [94]:
rfc= RandomForestClassifier(random_state=42)
scores = cross_val_score(rfc, X, y, cv=5)
print(f"Cross-validation accuracy: {scores.mean():.3f} (+/- {scores.std():.3f})")

rfc.fit(X, y)

Cross-validation accuracy: 0.384 (+/- 0.037)


RandomForestClassifier(random_state=42)

In [95]:
pred = rfc.predict(X2)

In [96]:
pred

array([1., 3., 2., 4., 2., 3., 1., 4., 3., 2., 2., 3., 2., 1., 2., 1., 4.,
       3., 3., 3., 3., 4., 3., 2., 3., 3., 1., 1., 4., 2., 4., 2., 2., 2.,
       1., 1., 3., 4., 4., 1., 4., 3., 1., 1., 4., 2., 2., 1., 1., 3., 2.,
       3., 1., 4., 1., 1., 2., 3., 4., 1., 4., 3., 1., 1., 4., 2., 3., 3.,
       3., 1., 1., 3., 1., 1., 2., 4., 2., 1., 4., 1., 2., 3., 3., 1., 1.,
       1., 1., 3., 1., 3., 3., 1., 3., 1., 2., 4., 1., 1., 1., 1., 1., 1.,
       2., 4., 4., 1., 4., 2., 4., 1., 1., 1., 1., 4., 1., 3., 2., 2., 4.,
       4., 1., 3., 1., 1., 4., 2., 2., 4., 4., 1., 4., 3., 1., 2., 1., 4.,
       3., 4., 2., 3., 3., 1., 2., 3., 2., 3., 3., 1., 4., 1., 4., 4., 4.,
       1., 1., 1., 3., 2., 3., 3., 2., 1., 1., 4., 2., 1., 2., 1., 4., 2.,
       3., 2., 4., 4., 2., 4., 1., 4., 3., 1., 2., 3., 4., 1., 4., 4., 4.,
       4., 4., 1., 4., 4., 1., 1., 3., 2., 3., 1., 3., 2., 2., 3., 4., 3.,
       3., 1., 4., 4., 1., 3., 4., 4., 3., 4., 3., 4., 1., 1., 3., 3., 4.,
       1., 3., 1., 2., 1.

In [104]:
import pandas as pd

In [107]:
len(rfc.feature_importances_)

128

In [114]:
importances = rfc.feature_importances_

importances_reshaped = importances.reshape(64, 2)

channel_importance = importances_reshaped.mean(axis=1)

# create dataframe
feature_importance = pd.DataFrame({
    'Channel': np.arange(1, 65),
    'Importance': channel_importance
}).sort_values('Importance', ascending=False)

print(feature_importance.head(10))

    Channel  Importance
31       32    0.018824
55       56    0.012352
30       31    0.012197
29       30    0.011765
45       46    0.010508
10       11    0.010162
43       44    0.009852
54       55    0.009478
26       27    0.009433
51       52    0.009368


In [ ]:
'''USING COMMON AVERAGE REFERENCE (CAR) METHOD
     signal = signal - mean(singal)
     The main idea being to remove the common shared noise
'''

In [115]:
X_train = []
y_train = []

for file in sorted(all_files)[:6]:
    file_path = os.path.join('/kaggle/working/Data', file)
    data = loadmat(file_path)

    signal = data['signal']
    trial = data['trial']
    Feedback = data['Feedback']
    TargetCode = data['TargetCode']
    # TargetCode 0 ->
    unique_trials = np.unique(trial[(TargetCode>0).flatten()])
    n_trials = len(unique_trials)
    signal_car = signal - signal.mean(axis=1, keepdims=True)
    
    Xx = np.zeros((n_trials, 128))
    yy = np.zeros(n_trials)
    for i, trial_num in enumerate(unique_trials):
        trial_mask = (trial== trial_num) & (Feedback==1)
        trial_signal = signal_car[trial_mask.flatten(), :]

        mu = []
        beta = []
        for ch in range(64): #64 channels
            channel_signal = trial_signal[:, ch]
            fft_values = np.fft.rfft(channel_signal)
            fft_freq = np.fft.rfftfreq(len(channel_signal), d=1/160) #160 Hz 
            # power = amplitude^2
            # Mu range 8-12 Hz, 
            mu_idx = (fft_freq >= 8) & (fft_freq <= 12)
            mu_power = np.sum(np.abs(fft_values[mu_idx])**2)
            mu.append(np.log(mu_power + 1e-10))
            # Beta range 18-24 Hz
            beta_idx = (fft_freq >=18) & (fft_freq <=24)
            beta_power = np.sum(np.abs(fft_values[beta_idx])**2)
            beta.append(np.log(beta_power+1e-10))
        Xx[i, :] = np.concatenate([mu, beta])
        yy[i] = TargetCode[trial_mask.flatten()][0]
    X_train.append(Xx)
    y_train.append(yy)
X = np.vstack(X_train)
y = np.concatenate(y_train)
print(f"X shape: {X.shape}, y shape: {y.shape}")

/tmp/ipykernel_37/1789551530.py:39: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  yy[i] = TargetCode[trial_mask.flatten()][0]


X shape: (1152, 128), y shape: (1152,)


In [116]:
svc = SVC(kernel='rbf', C=1.0, gamma='scale')

scores = cross_val_score(svc, X, y, cv=5)
print(f"Cross-validation accuracy: {scores.mean():.3f} (+/- {scores.std():.3f})")

svc.fit(X, y)

Cross-validation accuracy: 0.508 (+/- 0.052)


SVC()

In [117]:
rfc2 = RandomForestClassifier(random_state=32)

scores = cross_val_score(rfc2, X, y, cv=5)
print(f"Cross-validation accuracy: {scores.mean():.3f} (+/- {scores.std():.3f})")
rfc2.fit(X, y)

Cross-validation accuracy: 0.485 (+/- 0.053)


RandomForestClassifier(random_state=32)

**There is an increase in accuracy for the Training set by using CAR METHOD**
<h3> WITHOUT CAR:</h3>
svc accuracy: 0.418 (+/- 0.018)<br>
rfc accuracy: 0.384 <br>

<h3>WITH CAR:</h3> 
svc accuracy:0.508 (+/- 0.052)<br>
rfc accuracy: 0.485 (+/- 0.053) <br>

' NOW COMMON SPATIAL PATTERNS (CSP)\nTO FIND THE OPTIMAL LINEAR COMBINATION OF CHANNELS THAT MAXIMIZES \nVARIANCE BETWEEN CHANNELS \n\nCSP DOESNT WORK WITH CONTINUOUS SIGNALS, IT WORKS ON TRIALS'

In [ ]:
""" NOW COMMON SPATIAL PATTERNS (CSP)
TO FIND THE OPTIMAL LINEAR COMBINATION OF CHANNELS THAT MAXIMIZES 
VARIANCE BETWEEN classes 

CSP DOESNT WORK WITH CONTINUOUS SIGNALS, IT WORKS ON TRIALS"""

In [119]:
from mne.decoding import CSP
from scipy.signal import butter, filtfilt

In [121]:
def bandpass_filter(signal, lowcut=8, highcut=30, fs=160):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(4, [low, high], btype='band')
    return filtfilt(b, a, signal, axis=0)

In [122]:
def extract_trials_for_csp(data):
    signal = data['signal']
    trial = data['trial']
    Feedback = data['Feedback']
    TargetCode = data['TargetCode']
    
    unique_trials = np.unique(trial[(TargetCode > 0).flatten()])
    
    trials_list = []
    labels_list = []
    
    for trial_num in unique_trials:
        trial_mask = (trial == trial_num) & (Feedback == 1)
        trial_signal = signal[trial_mask.flatten(), :]  # (n_samples, 64)
        
        # Bandpass filter
        trial_filtered = bandpass_filter(trial_signal)
        
        # Transpose to (n_channels, n_samples) for MNE
        trials_list.append(trial_filtered.T)
        labels_list.append(TargetCode[trial_mask.flatten()][0])
    
    # Pad trials to same length (CSP needs equal length)
    max_len = max([t.shape[1] for t in trials_list])
    trials_padded = []
    for t in trials_list:
        if t.shape[1] < max_len:
            pad = np.zeros((t.shape[0], max_len - t.shape[1]))
            t = np.concatenate([t, pad], axis=1)
        trials_padded.append(t)
    
    X_trials = np.array(trials_padded)  # (n_trials, n_channels, n_samples)
    y_trials = np.array(labels_list)
    
    return X_trials, y_trials

In [128]:
total_XTrials, total_yTrials= [], []
for file in sorted(all_files)[:6]:
    file_path = os.path.join('/kaggle/working/Data', file)
    data = loadmat(file_path)
    X_trials, y_trials = extract_trials_for_csp(data)
    total_XTrials.append(X_trials)
    total_yTrials.append(y_trials)
X = np.concatenate(total_XTrials, axis=0)
y = np.concatenate(total_yTrials)

In [133]:
y = y.ravel()

In [134]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score

csp = CSP(n_components=4, reg=None, log=True)
X_csp = csp.fit_transform(X, y)  # (n_trials, 4) features

print(f"CSP features shape: {X_csp.shape}") 

# Now train classifier on X_csp
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Cross-validate
scores = cross_val_score(clf, X_csp, y, cv=5)
print(f"CSP + RF accuracy: {scores.mean():.3f} (+/- {scores.std():.3f})")

# Fit on all data
clf.fit(X_csp, y)

Computing rank from data with rank=None
    Using tolerance 3.7e+04 (2.2e-16 eps * 64 dim * 2.6e+18  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Estimating class=4 covariance using EMPIRICAL
Done.
CSP features shape: (1152, 4)
CSP + RF accuracy: 0.631 (+/- 0.043)


RandomForestClassifier(random_state=42)